In [1]:
import glob
import tensorflow as tf
from pathlib import Path
import math
from IPython.display import display
def datasetImageFolder(start_path,split=None):
    filePath = getFilePaths(start_path,["jpg","png","jpeg"])
    splits =  list(set(map(lambda x: x[x.find('\\',len(start_path)-1 )+1:x.find('\\',x.find('\\',len(start_path)-1 )+1) ] , filePath)))
    
    allSplits = []
    for i in range(len(splits)):
        rs = list(map(lambda x : x if splits[i] in x else None, filePath))
        result = [x for x in rs if x]
        allSplits.append(result)
    
    datasets = dict()
    for i in range(len(allSplits)):
        images  = tf.data.Dataset.from_tensor_slices(allSplits[i])
        datasets[splits[i]] = images
    
    return datasets

def getFilePaths(start_path,extensions= None):
    paths = []
    if(extensions is not None):
        for extension in extensions:
            paths.extend(list(Path(start_path).rglob("*." + extension)))
    
    else:
        path.extend(list(Path(start_path).rglob()))
    
    return [str(x) for x in paths if x.is_file()]
    
    
def getLabelFromFilePathTF(file_path):
    splits = tf.strings.split(file_path,'\\')
    label = splits[len(splits)-2]
    return label

def getLabelFromFilePath(file_path):
    lastIndex = file_path.rfind('\\')
    firstIndex = file_path.rfind('\\',0,lastIndex)+1
    label =file_path[firstIndex:lastIndex]
    
    return label

def getAllLabels(start_path,generateUndefined=False):
    paths = getFilePaths(start_path,["jpg","png","jpeg"])
    distinctLabels = []
    if (generateUndefined):
        distinctLabels.append('Undefined')
    
    distinctLabels.extend(list(set(map(lambda x: getLabelFromFilePath(x) , paths))))
        
    return distinctLabels

def generateOneHotEncodeDict(labels):
    indices = [x for x in range(len(labels))]
    one_hots = tf.one_hot(indices,len(labels),dtype=tf.uint8)
    
    one_hot_dict = dict()
    for i in range(len(one_hots)):
        one_hot_dict[labels[i]] = one_hots[i]
        
    return one_hot_dict
    
def generateBinaryEncoding(labels):
    indices = [x for x in range(len(labels))]
    n_bits = int(math.log(len(labels),2)) + 1 
    binaryEncode = []
    for index in indices:
        encoding = []
        for i in range(n_bits):
            encoding.append( (index >> i) & 1)
        encoding.reverse()
        binaryEncode.append(encoding)
    
    
    binaryEncodeTensors = [tf.convert_to_tensor(x,dtype=tf.uint8) for x in binaryEncode]
    
    binary_encode_dict = dict()
    for i in range(len(labels)):
        binary_encode_dict[labels[i]] = binaryEncodeTensors[i]
        
    return binary_encode_dict

def loadBestSavedModel(start_path):        
    models = getFilePaths(start_path,['h5'])
    
    bestModel = -1
    modelToLoad = ""
    
    for modelPath in models:
        firstIndex = modelPath.find(".",modelPath.find(".")+1)+1
        
        
        modelNumber = modelPath[firstIndex:modelPath.rfind(".")]
        
        if(float(modelNumber) > bestModel ):
            bestModel = float(modelNumber)
            modelToLoad = modelPath
    
    print(modelToLoad)
    #if len(modelToLoad) > 0:
       #return keras.models.load_model(modelToLoad)
    
    
    
    return modelToLoad
    
    


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D,Input,Dense,Flatten,LeakyReLU,MaxPooling2D,Dropout,Softmax,ReLU
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2
import tensorflow_datasets as tfds
from PIL import Image
from keras import metrics

In [3]:
def vgg(filters=64,n_Class=1000,shape=(224,224,3)):
    
    def convBlock(input_tensor,filters=filters):
        x = Conv2D(filters,kernel_size=3,strides=1,padding='same',kernel_initializer=tf.keras.initializers.HeUniform(),kernel_regularizer=l2(0.0001))(input_tensor)
        x = LeakyReLU(alpha=0.2)(x)
        return x
        
         
    input_tensor = Input(shape=shape)
    
    x = convBlock(input_tensor,filters)
    x = convBlock(x,filters)
    x = MaxPooling2D(pool_size=(2,2),strides=2)(x)
    
    x = convBlock(x,filters*2)
    x = convBlock(x,filters*2)
    x = MaxPooling2D(pool_size=(2,2),strides=2)(x)
    
    x = convBlock(x,filters*4)
    x = convBlock(x,filters*4)
    x = convBlock(x,filters*4)
    x = convBlock(x,filters*4)
    x = MaxPooling2D(pool_size=(2,2),strides=2)(x)
    
    x = convBlock(x,filters*8)
    x = convBlock(x,filters*8)
    x = convBlock(x,filters*8)
    x = convBlock(x,filters*8)
    x = MaxPooling2D(pool_size=(2,2),strides=2)(x)
    
    x = convBlock(x,filters*8)
    x = convBlock(x,filters*8)
    x = convBlock(x,filters*8)
    x = convBlock(x,filters*8)
    x = MaxPooling2D(pool_size=(2,2),strides=2)(x)
    
    x = Flatten()(x)
    
    x = Dense(4096,kernel_initializer=tf.keras.initializers.HeUniform())(x)
    x = Dropout(0.5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Dense(4096,kernel_initializer=tf.keras.initializers.HeUniform())(x)
    x = Dropout(0.5)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Dense(n_Class)(x)
    x = Softmax(axis=-1)(x)
    
    return Model(inputs=input_tensor,outputs=x)

def preprocessImageVgg(image):
    img = tf.image.random_crop(image,size=[224,224,3])
    img /= 127.5
    img -= 1.
    
    return img
    
def getDatasetFromImageFolder(imageFolder,batchSize=32,n_epochs=100):
   
    def preprocessVgg(ds):
        
        ds = ds.map(lambda image ,label: (preprocessImageVgg(image) , tf.reshape(label ,(270,))) , num_parallel_calls=tf.data.AUTOTUNE)
        return ds
    
    def augmentDataset(ds):
        ds = ds.map(lambda x ,y: (tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)) ,y), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.map(lambda x ,y: (tf.image.random_flip_left_right(x),y), num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.map(lambda x ,y: (tf.image.random_flip_up_down(x),y), num_parallel_calls=tf.data.AUTOTUNE)
        return ds
    

    def process_img(img):
        img = tf.image.decode_jpeg(img, channels=3) 
        #img = tf.image.convert_image_dtype(img, tf.float32) 
        return tf.image.resize(img, [224, 224])
    
    def combine_images_labels(file_path: tf.Tensor):
        img = tf.io.read_file(file_path)
        img = process_img(img)
        label = getLabelFromFilePathTF(file_path)
        
        labelEncoded = encodingLabelDict[label.numpy().decode('utf-8')]
        
        return img, labelEncoded
        
        
    def prepareDataset(ds, training=False):
        ds = ds.shuffle(buffer_size=10000)        
        ds = ds.map(lambda x : tf.py_function(func=combine_images_labels,
                        inp=[x], Tout=(tf.float32,tf.uint8)),
                        num_parallel_calls=tf.data.AUTOTUNE)
        
        ds = preprocessVgg(ds)
        if(training):
            ds = augmentDataset(ds)
            
        ds = ds.repeat()
        ds = ds.batch(batchSize)
        
        ds = ds.apply(tf.data.experimental.copy_to_device('/gpu:0'))
        ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
        
        return ds
        
    datasets = datasetImageFolder(imageFolder)
    
    
    dsTrain, dsTest, dsValid = datasets['train'] ,datasets['test'],datasets['valid']
    
    #encodingLabelDict = generateBinaryEncoding(getAllLabels(imageFolder))
    encodingLabelDict = generateOneHotEncodeDict(getAllLabels(imageFolder))
    steps_per_epoch_train =  dsTrain.cardinality().numpy() // batchSize
    steps_per_epoch_valid =  dsValid.cardinality().numpy() // batchSize
    labelDict = generateBinaryEncoding(getAllLabels(imageFolder))  
    
    dsTrain = prepareDataset(dsTrain,training=True)
    
    dsTest = prepareDataset(dsTest)
    dsValid = prepareDataset(dsValid)
    
    return dsTrain,dsTest,dsValid,steps_per_epoch_train,steps_per_epoch_valid

def schedule(epoch,lr):
        return lr / max(10 * int(epoch/20) ,1)
    
    
def trainVgg(model,trainDataset,validDataset,epochs=100,verbose=1,steps_per_epoch=None,steps_per_epoch_valid=None):
    my_callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10),
        tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}.{val_loss:02f}.h5', monitor='val_loss',  verbose=0, save_best_only=True),
        tf.keras.callbacks.TensorBoard(log_dir='./logs'),
        tf.keras.callbacks.LearningRateScheduler(schedule, verbose=0)
    ]
    
    if steps_per_epoch is not None and steps_per_epoch_valid is not None :
        hist = model.fit(x=trainDataset,validation_data=validDataset,steps_per_epoch=steps_per_epoch,epochs=epochs, callbacks=my_callbacks , verbose=verbose,validation_steps=steps_per_epoch_valid,initial_epoch=15)
    else:
        hist = model.fit(trainDataset,validation_data=validDataset,callbacks=my_callbacks,verbose=verbose)
    
    
    
    return hist

In [4]:
n_labels = len( getAllLabels('data/'))
n_Class = int(math.log(n_labels,2))+1
print( n_labels)


270


In [5]:
#model_vgg = vgg(filters=64,n_Class=n_labels,shape=(224,224,3))


In [6]:
dsTrain,dsTest,dsValid ,stepTrain,steps_per_epoch_valid = getDatasetFromImageFolder('data/',batchSize=16)

for image, label in dsTrain.take(1):
    print(label.numpy().shape)

(16, 270)


In [7]:
print(stepTrain)
print(steps_per_epoch_valid)
print(dsTrain.element_spec)

#checkSavedModel("./")

2407
84
(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 270), dtype=tf.uint8, name=None))


In [9]:
def compileVgg(model ,learning_rate=1e-2):
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate,clipnorm=1,nesterov=True),loss='categorical_crossentropy',metrics=[metrics.categorical_accuracy])

    
model = loadBestSavedModel(".")    

model_vgg = tf.keras.models.load_model(model)
#compileVgg(model_vgg,learning_rate=1e-2)
#model_vgg.load_weights(model)

hist = trainVgg(model_vgg,trainDataset=dsTrain,validDataset=dsValid,epochs=100,verbose=1,steps_per_epoch=stepTrain,steps_per_epoch_valid=steps_per_epoch_valid)

model.15.3.005072.h5
Epoch 16/100
  98/2407 [>.............................] - ETA: 14:58 - loss: 6.6402 - categorical_accuracy: 0.0300

KeyboardInterrupt: 